In [7]:
# setting virtual environments
# - env/
# python -m venv ./env or python3.11 -m venv ./env
# .gitignore > env/
# source env/bin/activate
# deactivate
# pip install -r requirements.txt
# insert api key, .env

# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
)  # ChatPromptTemplate은  template을 메시지로부터 만들고 PromptTemplate 은 그냥 string을 이용해서 template을 만듦.

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1  # 창의력을 결정. 값이 낮을수록 창의성 저하.
    # max_tokens= 최대 토큰값 정함.
)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

# a = llm.predict("How many planets are there?") # text-davinci language model호출.
# b = chat.predict("How many planets are there?")  # gpt-3.5-turbo 호출. 훨씬 저렴함.
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)
# a
# b

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [8]:
# from langchain.schema import (
#     HumanMessage,
#     AIMessage,
#     SystemMessage,
# )  # SystemMessage:LLM에 설정제공 위한 메시지


# messages = [
#     SystemMessage(
#         content="You are a geography expert. And you only reply in {language}.",
#     ),
#     AIMessage(content="Ciao, mi chiamo {name}!"),
#     HumanMessage(
#         content="What is the distance between {country_a} and {country_b}. Also, what is your name?",
#     ),
# ]
# 위 messages를 아래와 같이 튜플로

template = ChatPromptTemplate.from_messages(
    [  # ChatPromptTemplate 이용해서 커스텀하기
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)


# chat.predict_messages(messages)
prompt = template.format_messages(  # 변수들을 검증
    language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate

# Output Parser가 필요한 이유는 LLM의 Response을 변형해야 할 때가 있기 때문.
# 아래에서 Response를 list로 변환시키는 것을 만들어 볼 거임.
# LLM은 항상 텍스트로만 응답함.
# 그래서 그 응답을 데이터베이스에 넣을 형태 또는 dictionary, tuple에 저장할 수 있게 변형(transform) 시킬 수 있다. 
# 이것이 바로 Output Parser가 필요한 이유.



chat = ChatOpenAI(temperature=0.1)


In [10]:
from langchain.schema import BaseOutputParser


class CommaOutputPaser(BaseOutputParser):  # output parser는 pares라는 메서드를 꼭 구현해야 함.
    def parse(self, text):  # text매개변수를 통해 전체 텍스트를 입력 받을거임.
        # return text.strip().split(".")  # strip은 text의 앞뒤 공백(space)를 잘라내 줌.
        items = text.strip().split(",")
        return list(map(str.strip,items))


p = CommaOutputPaser()
p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [13]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generation machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
    
])

# prompt = template.format_messages(max_items=10, question="What are the planets?")
prompt = template.format_messages(max_items=10, question="What are the colors?")
chat.predict_messages(prompt)

AIMessage(content='Red, orange, yellow, green, blue, indigo, violet, black, white, gray')

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generation machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(max_items=10, question="What are the planets?")
# prompt = template.format_messages(max_items=10, question="What are the colors?")
chat.predict_messages(prompt)

AIMessage(content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune')

In [15]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generation machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(max_items=10, question="What are the planets?")
# prompt = template.format_messages(max_items=10, question="What are the colors?")
chat.predict_messages(prompt)

AIMessage(content='mercury, venus, earth, mars, jupiter, saturn, uranus, neptune, pluto')

아래는 outputParser를 가져와 적용

In [17]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generation machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# prompt = template.format_messages(max_items=10, question="What are the planets?")
prompt = template.format_messages(max_items=10, question="What are the colors?")
result = chat.predict_messages(prompt)

p = CommaOutputPaser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

위와 같이 하지 않고 아래와 같이 간단하게 구현할 수 있다. 이것이 LangChain 이다.

사실은 LangChain이 

template 으로 .format_messages를 호출하고 
chat 으로 chat.predict도 호출하고 
CommaOutputPaser 으로 parse를 호출해 준 것임. 아래 코드를 보라. invoke 호출을 통해서 이 모든 걸 이룸. LangChain expression language.

In [18]:
chain = template | chat | CommaOutputPaser()

chain.invoke({
    "max_items" : 5,
    "question":"What are the pokemons?"
    
}) # invokde 메서드의 입력값으로는 dictionary 타입이 들어가야 함. 

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [26]:
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler(),],
)  # streaming=True model의 응답이 생성되는 걸 볼 수 있게 해줌.


chef_promt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to fine ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_promt | chat

In [27]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't konw how to replace it."),
    ("human", "{recipe}")
])



In [28]:
veg_chain = veg_chef_prompt | chat
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine" : "indian"
}) 

# veg_chain.invoke({
#     "recipe" : "chatmodel"
# })

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.

2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.

3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.

4. Add the chicken pieces to the pan and cook until they are lightly brow

AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter or vegetable oil\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream or coconut cream\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter or vegetable oil in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixtu